In [1]:
# !git clone https://github.com/RGU-Computing/DisCERN-XAI
!git clone https://github.com/pedramsalimi/DisCERN-XAI
!pip install lime
!pip install shap

Cloning into 'DisCERN-XAI'...
remote: Enumerating objects: 108, done.
remote: Counting objects: 100% (108/108), done.
remote: Compressing objects: 100% (77/77), done.
remote: Total 108 (delta 50), reused 62 (delta 22), pack-reused 0
Receiving objects: 100% (108/108), 1.54 MiB | 12.66 MiB/s, done.
Resolving deltas: 100% (50/50), done.
     |████████████████████████████████| 275 kB 28.6 MB/s 
  Created wheel for lime: filename=lime-0.2.0.1-py3-none-any.whl size=283857 sha256=59ee5649c2c1d0a7f49fb949bb646a526d441be0b89f014349f2ba55097787bc
  Stored in directory: /root/.cache/pip/wheels/ca/cb/e5/ac701e12d365a08917bf4c6171c0961bc880a8181359c66aa7
Successfully built lime
     |████████████████████████████████| 356 kB 33.3 MB/s 
  Created wheel for shap: filename=shap-0.39.0-cp37-cp37m-linux_x86_64.whl size=491649 sha256=36f26b15cee212cd5f5a09c09c46c660b24610a1644dde70068218dcb620cfc3
  Stored in directory: /root/.cache/pip/wheels/ca/25/8f/6ae5df62c32651cd719e972e738a8aaa4a87414c4d2b14c9c0
Su

In [2]:
cd DisCERN-XAI/

/content/DisCERN-XAI


In [3]:
!pip install -r requirements.txt
!pip install -e .

     |████████████████████████████████| 2.9 MB 23.2 MB/s 
     |████████████████████████████████| 1.2 MB 54.1 MB/s 
     |████████████████████████████████| 56 kB 5.0 MB/s 
     |████████████████████████████████| 895 kB 39.7 MB/s 
     |████████████████████████████████| 3.3 MB 29.9 MB/s 
     |████████████████████████████████| 636 kB 45.6 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Obtaining file:///content/DisCERN-XAI
  Running setup.py develop for discern-xai


In [4]:
!mkdir discern/NLG_model

In [5]:
!wget https://www.dropbox.com/s/15uwbwv00oubrf7/model.pt?dl=0

--2021-10-07 10:31:07--  https://www.dropbox.com/s/15uwbwv00oubrf7/model.pt?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.4.18, 2620:100:601c:18::a27d:612
Connecting to www.dropbox.com (www.dropbox.com)|162.125.4.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/15uwbwv00oubrf7/model.pt [following]
--2021-10-07 10:31:07--  https://www.dropbox.com/s/raw/15uwbwv00oubrf7/model.pt
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc457f63436c52d9e371046e4c9c.dl.dropboxusercontent.com/cd/0/inline/BXnlL1-3JKI3rTyDPryw2WmcOvEvG4PUAEwq2j5BOVA4u0SEby9QECPsTVQWdUERJywNjdMEk3hd0zUGy5YOAvHS0TAFOBCld4dfJvH55h-pm-zaBKAljLp1H40DE81ANmCLEFusW_ktQyg9-fZCk1xV/file# [following]
--2021-10-07 10:31:07--  https://uc457f63436c52d9e371046e4c9c.dl.dropboxusercontent.com/cd/0/inline/BXnlL1-3JKI3rTyDPryw2WmcOvEvG4PUAEwq2j5BOVA4u0SEby9QECPsTVQWdUERJywNjdMEk3hd0zUGy5YOAvHS0TAFOBCld

In [6]:
mv /content/DisCERN-XAI/model.pt?dl=0 /content/DisCERN-XAI/discern/NLG_model/model.pt

In [16]:
from discern.discern_tabular import DisCERNTabular
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
svm = SVC(probability=True)
def test_cancer_risk(q,label,d_label):
    data_df = pd.read_csv('/content/DisCERN-XAI/tests/lung_cancer.csv')
    data_df = data_df.replace({'Level': {'Low': 0, 'Medium': 1, 'High': 2}})
    data_df = data_df.replace({'Gender': {2: 0}})
    data_df = data_df.replace({'Alcohol use': {2: 0}})
    data_df = data_df.replace({'Dust Allergy': {2: 0}})
    data_df = data_df.replace({'Smoking': {2: 0}})
    data_df = data_df.replace({'Chest Pain': {2: 0}})
    data_df = data_df.replace({'Fatigue': {2: 0}})
    data_df = data_df.replace({'Shortness of Breath': {2: 0}})
    data_df = data_df.replace({'Wheezing': {2: 0}})
    data_df = data_df.replace({'Swallowing Difficulty': {2: 0}})
    data_df = data_df.replace({'Cough': {2: 0}})
    data_df = data_df.replace({'chronic Lung Disease': {2: 0}})
    # print("Reading data complete!",data_df.columns)
    discern = DisCERNTabular(svm, 'LIME', 'Q')

    df = data_df.copy()
    x = df.loc[:, df.columns != 'Level'].values
    y = df['Level'].values
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=1)
    scaler = MinMaxScaler()
    x_train= scaler.fit_transform(x_train)
    x_test = scaler.transform(x_test)
    svm.fit(x_train, y_train)
    x_test = scaler.transform([q])
    y_test = svm.predict(x_test)

    cat_indices = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]

    sparsity = []
    proximity = []
    discern = DisCERNTabular(svm, 'LIME', 'Q')
    discern.init_data(x_train, y_train, [c for c in df.columns if c!='Level'], ['Low', 'Medium', 'High'], cat_feature_indices=cat_indices)
    cf, s, p = discern.find_cf(x_test[0], {'Low': 0, 'Medium': 1, 'High': 2}[label], desired_class=d_label)
    sparsity.append(s)
    proximity.append(p)
    _sparsity = sum(sparsity)/len(sparsity)
    _proximity = sum(proximity)/(len(proximity)*_sparsity)
   
    x = scaler.inverse_transform(x_test)
    c = scaler.inverse_transform([cf])
    for i in range(len(data_df.columns)-1):
      if c[0][i] == x[0][i]:
        pass
      else:
        print("{} should be changed from {} to {}".format(data_df.columns[i],x[0][i],c[0][i]))
        cls = list(df.columns)
        x=pd.DataFrame(x,columns=[c for c in df.columns if c!='Level'])
        c=pd.DataFrame(c,columns=[c for c in df.columns if c!='Level'])
        x['Level'] = y_test
        c['Level'] = 1 # Temporary

        x=x.to_dict(orient="index")[0]
        c=c.to_dict(orient="index")[0]
        discern.show_cf(x, c)
        break

In [8]:
data_df = pd.read_csv('/content/DisCERN-XAI/tests/lung_cancer.csv')
data_df.columns

Index(['Age', 'Gender', 'Alcohol use', 'Dust Allergy', 'chronic Lung Disease',
       'Smoking', 'Chest Pain', 'Fatigue', 'Shortness of Breath', 'Wheezing',
       'Swallowing Difficulty', 'Cough', 'Level'],
      dtype='object')

In [9]:
features = ['Age', 'Gender', 'Alcohol use', 'Dust Allergy', 'chronic Lung Disease',
            'Smoking', 'Chest Pain', 'Fatigue', 'Shortness of Breath', 'Wheezing',
            'Swallowing Difficulty', 'Cough', 'Level','Target Level']
data_dic = {}
for i in features:
  if i=="Level" or i=="Target Level":
    data_dic[i] = list(set(data_df.Level))
  else:
    data_dic[i] = [data_df.describe()[i]["min"], data_df.describe()[i]["max"]]

In [18]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
import numpy as np

tokenizer = AutoTokenizer.from_pretrained("microsoft/DialoGPT-medium")
model = AutoModelForCausalLM.from_pretrained("microsoft/DialoGPT-medium")

features = ['Age', 'Gender', 'Alcohol use', 'Dust Allergy', 'chronic Lung Disease',
            'Smoking', 'Chest Pain', 'Fatigue', 'Shortness of Breath', 'Wheezing',
            'Swallowing Difficulty', 'Cough', 'Level', 'Target Level']
exit = ["Thanks","Goodbye","goodbye","Bye","bye"]

for step in range(5):
  new_user_input_ids = tokenizer.encode(input(">> User:") + tokenizer.eos_token, return_tensors='pt')
  print(tokenizer.decode(new_user_input_ids[0], skip_special_tokens=True))
  bot_input_ids = torch.cat([chat_history_ids, new_user_input_ids], dim=-1) if step > 0 else new_user_input_ids
  chat_history_ids = model.generate(bot_input_ids, max_length=1000, pad_token_id=tokenizer.eos_token_id)
  print("DialoGPT: {}".format(tokenizer.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)))
  if "help" in tokenizer.decode(new_user_input_ids[0], skip_special_tokens=True):
    info = []
    lvl = ""
    trg = ""
    for i in range(len(data_dic)):
      if features[i]=="Level":
        lvl = input(f"Please give info about your {features[i]} status in a following range: {data_dic[features[i]]}")
      elif features[i]=="Target Level":
        trg = input(f"Please give info about your {features[i]} status in a following range: {data_dic[features[i]]}")
      else:
        info.append(float(input(f"Please give info about your {features[i]} status in a following range: {data_dic[features[i]]}")))
    test_cancer_risk(np.array(info), lvl, trg)
  if tokenizer.decode(new_user_input_ids[0], skip_special_tokens=True) in exit:
    print("Have a good day, bye.")
    break

>> User:Hello
Hello
DialoGPT: Hello! :D
>> User:how are you?
how are you?
DialoGPT: I'm good, how are you?
>> User:Thanks, can you help me in some sort of decision making?
Thanks, can you help me in some sort of decision making?
DialoGPT: I can try, but I'm not sure if I can help you.
Please give info about your Age status in a following range: [14.0, 87.0]15
Please give info about your Gender status in a following range: [1.0, 2.0]1
Please give info about your Alcohol use status in a following range: [1.0, 2.0]2
Please give info about your Dust Allergy status in a following range: [1.0, 2.0]1
Please give info about your chronic Lung Disease status in a following range: [1.0, 2.0]1
Please give info about your Smoking status in a following range: [1.0, 2.0]1
Please give info about your Chest Pain status in a following range: [1.0, 2.0]1
Please give info about your Fatigue status in a following range: [1.0, 2.0]1
Please give info about your Shortness of Breath status in a following range